# Linear Regression ML 

RidgeCV( df[['Detector', 'Optics', 'Source']] , df['R_free'] )


1. Transformer to select Detectors, Optics, Source
2. OneHot Encode All
3. RidgeCV? RandomForest?

In [1]:
import pandas as pd
import numpy as np

In [41]:
df = pd.read_feather('/Users/jesse/TDI_bootcamp/capstone_PDB/dataframe_feather/2_detclean_dataframe.feather')

In [42]:
#Drop R Nulls
df = df[df['R_free'] != "NULL"]
df = df[df['R_value'] != "NULL"]
df = df[df['I_sigma'] != "NULL"]


#Cast as floats
df = df.astype({'R_free': 'float', 'R_value': 'float', 'I_sigma': 'float'})

In [43]:
df.count()

level_0         107723
index           107723
Resolution      107723
Completeness    107723
I_sigma         107723
R_value         107723
R_free          107723
Detector        107723
Det_type        107723
Optics          107723
dtype: int64

In [56]:
det_df = (df.groupby('Det_type')
 .agg({'R_free': 'mean', "R_value": 'mean', "I_sigma": 'mean', "index": 'count'})
 .sort_values(["index"])
)
 
det_df.loc[det_df['index'] > 100]

,R_free,R_value,I_sigma,index
Det_type,,,,
CCD (QUANTUM 315),0.233219,0.192232,15.850935,107
CCD (Q315R),0.226405,0.185237,17.048291,117
PIXEL (PILATUS),0.231627,0.193378,14.865616,146
CCD (MX-225),0.230511,0.188841,14.881991,211
CCD (QUANTUM 315R),0.231006,0.190639,15.120195,307
CMOS,0.230980,0.195527,16.944297,400
CCD; CCD,0.239470,0.200892,17.894104,529
PIXEL (PILATUS 6M),0.227821,0.190851,13.962473,558
AREA DETECTOR,0.234935,0.193745,16.323913,783


In [21]:
df['Optics'].unique()

array(['NULL', '???', '3.0 UNDULATOR C(111)', 'ROSENBAUM ROCK VERTICAL',
       'BERYLLIUM LENSES', 'MIRRORS', 'MIRROR', 'VARIMAX HF',
       'OSMIC VARIMAX HF ARCSEC', 'RH COATED COLLIMATOR',
       'MIRRORS (VFM,HFM)', 'VARIMAX HIGHFLUX',
       'RH COATED COLLIMATING MIRRORS, K', 'BERRYLIUM LENSES',
       'KIRKPATRICK-BAEZ MIRRORS', 'BE', 'PT/RH MIRRORS', 'KB MIRRORS',
       'KB-MIRROR SYSTEM', 'COMPOUND REFRACTIVE LENSES',
       'RH COATED COLLIMATING MIRROR,', 'FLAT BENT COLIMATOR RH COVERED',
       'CRL', '3.0G UNDULATOR, DIAMOND (111)', 'FLAT SI RH COATED M0,',
       'SI MIRROR', 'MIRROR FOCUSING', 'SI 111. ROSENBAUM-ROCK DOUBLE',
       'BERYIILUM LENSES', 'EIGER2 XE 16M', 'MONOCHROMATOR', 'UNDULATOR',
       'MICRO-FOCUSED BEAM OF 10 MICRO M',
       'FLAT BENT COLLIMATING RH COATED',
       'TWO DIMENSIONAL FOCUSING MIRROR', 'VERTICALLY COLLIMATING MIRROR',
       'KB FOCUSING MIRRORS', 'BENT COLLIMATING MIRROR AND',
       'DCM SI(111)', 'KIRKPATRICK-BAEZ PAIR OF BI',
 

In [57]:
from sklearn.preprocessing import OneHotEncoder


ohe = OneHotEncoder()
ohe.fit_transform(df[['Detector', 'Det_type', 'Optics']]) 

<107723x2009 sparse matrix of type '<class 'numpy.float64'>'
	with 323169 stored elements in Compressed Sparse Row format>

In [59]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import RidgeCV

categorical_columns = ['Detector', 'Det_type', 'Optics']

est = Pipeline([
    ('categorical', OneHotEncoder()),
    ('regressor', RidgeCV())
])

est.fit(df[categorical_columns], df['I_sigma'])

Pipeline(memory=None,
         steps=[('categorical',
                 OneHotEncoder(categorical_features=None, categories=None,
                               drop=None, dtype=<class 'numpy.float64'>,
                               handle_unknown='error', n_values=None,
                               sparse=True)),
                ('regressor',
                 RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=None,
                         fit_intercept=True, gcv_mode=None, normalize=False,
                         scoring=None, store_cv_values=False))],
         verbose=False)

In [61]:
est.score(df[categorical_columns], df['I_sigma'])

0.003128282560099338